In [1]:

# Let's say we are deling with power outages and we need roll trucks to various sites around the south and 
# mid-west United States to restore power at various sites. looking at latitude and longitude coordinates, we can 
# assign the truck closest to the outage. Also we need to manage schedules efficiently, so if 2 workers are 
# coming close to 8 hours, I won't assign them, but I'll pick two workers who have been working only 6 hours 
# so far today? Finally, I need to make sure the workers have the skills needed to handle the problems they 
# will deal with to restore power. So, if a truck is closer to the outage but the workers don't have the 
# right skills, I won't send this truck. Instead, I'll send a truck that is further away, but they have the right 
# skills to deal with the issue. 


In [3]:

import numpy as np

# Generate sample data
np.random.seed(42)
num_trucks = 5
num_outages = 10

truck_coordinates = np.random.rand(num_trucks, 2) * 100  # Random coordinates
outage_coordinates = np.random.rand(num_outages, 2) * 100  # Random coordinates

# Use labels for skills
skill_labels = ["Electrical", "Mechanical", "Communication"]
worker_skills = np.random.choice(skill_labels, size=(num_trucks, 3))

# Generate random hours worked by each worker
worker_hours_worked = np.random.randint(4, 8, size=num_trucks)

# Create a distance matrix
distance_matrix = np.linalg.norm(truck_coordinates[:, np.newaxis, :] - outage_coordinates, axis=2)

# Solve the assignment problem (closest truck to each outage)
assignment = np.argmin(distance_matrix, axis=0)

# Print optimal truck assignment with distance and hours worked
for outage_index, truck_index in enumerate(assignment):
    distance = distance_matrix[truck_index, outage_index]
    skills = ', '.join(worker_skills[truck_index])
    hours_worked = worker_hours_worked[truck_index]
    
    if hours_worked < 8:
        print(f"Outage {outage_index + 1}: Truck {truck_index + 1} ({skills}), Distance: {distance:.2f} miles away, Hours Worked: {hours_worked}")
    else:
        print(f"Outage {outage_index + 1}: Truck {truck_index + 1} ({skills}), Distance: {distance:.2f} miles away, Hours Worked: {hours_worked} (Overtime)")
        

Outage 1: Truck 4 (Mechanical, Electrical, Mechanical), Distance: 11.03 miles away, Hours Worked: 4
Outage 2: Truck 2 (Electrical, Communication, Mechanical), Distance: 39.92 miles away, Hours Worked: 6
Outage 3: Truck 3 (Electrical, Mechanical, Mechanical), Distance: 3.76 miles away, Hours Worked: 7
Outage 4: Truck 5 (Electrical, Mechanical, Communication), Distance: 34.89 miles away, Hours Worked: 7
Outage 5: Truck 3 (Electrical, Mechanical, Mechanical), Distance: 30.73 miles away, Hours Worked: 7
Outage 6: Truck 3 (Electrical, Mechanical, Mechanical), Distance: 45.61 miles away, Hours Worked: 7
Outage 7: Truck 3 (Electrical, Mechanical, Mechanical), Distance: 25.06 miles away, Hours Worked: 7
Outage 8: Truck 5 (Electrical, Mechanical, Communication), Distance: 16.43 miles away, Hours Worked: 7
Outage 9: Truck 3 (Electrical, Mechanical, Mechanical), Distance: 36.09 miles away, Hours Worked: 7
Outage 10: Truck 3 (Electrical, Mechanical, Mechanical), Distance: 44.99 miles away, Hours W

In [4]:

# The details described here have elements reminiscent of a Traveling Saleserson Problem, but rather than optimizing
# (minimizing) the path of the salesperson this problem is a little more complex because we have to do the following:
# 1) Assigning Trucks to Outages
# 2) Skills Matching
# 3) Working Hours Constraint


In [ ]:

# END...
